# Practice Project


This practice project focuses on data transformation and integration using PySpark. You will work with two datasets, perform various transformations such as adding columns, renaming columns, dropping unnecessary columns, joining dataframes, and finally, writing the results into both a Hive warehouse and an HDFS file system.


### Prerequisites

For this lab assignment, you will use wget, Python and Spark (PySpark). Therefore, it's essential to make sure that the below-specified libraries are installed in your lab environment or within Skills Network (SN) Labs.




In [1]:
# Installing required packages

!pip install wget pyspark findspark

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=a495a2b56c731fa6f8588b740a32f322fc834e3a9267b2315d9808c178d71c3c
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=59a95e01d292b0b866d45c3e38296bf238c4498fb1d44b67d26fbbe6ee97edd4
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built wget pyspark


#### Prework - Initiate the Spark Session


In [1]:
import findspark

findspark.init()

In [24]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the SparkContext.

from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession

from pyspark.sql.functions import year, quarter, to_date, date_format

from pyspark.sql.functions import col, when

In [3]:
# Creating a SparkContext object

sc = SparkContext.getOrCreate()

# Creating a Spark Session

spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

### Task 1: Load datasets into PySpark DataFrames

Download the datasets from the folloing links using `wget` and load it in a Spark Dataframe.

1. https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv  
2. https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset2.csv  


In [5]:
#download dataset using wget
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv

!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset2.csv


--2024-06-16 08:53:41--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4115 (4.0K) [text/csv]
Saving to: ‘dataset1.csv’

dataset1.csv        100%[===================>]   4.02K  --.-KB/s    in 0s      

2024-06-16 08:53:42 (1.20 GB/s) - ‘dataset1.csv’ saved [4115/4115]

--2024-06-16 08:53:42--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset2.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.10

In [4]:
#load the data into a pyspark dataframe
df1 = spark.read.csv("dataset1.csv",header=True, inferSchema=True)

df2 = spark.read.csv("dataset2.csv",header=True, inferSchema=True)


### Task 2: Display the schema of both dataframes

Display the schema of `df1` and `df2` to understand the structure of the datasets.


In [5]:
#print the schema of df1 and df2

print(df1.printSchema())

print(df2.printSchema())

root
 |-- customer_id: integer (nullable = true)
 |-- date_column: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- location: string (nullable = true)

None
root
 |-- customer_id: integer (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- value: integer (nullable = true)
 |-- notes: string (nullable = true)

None


In [6]:
# get the number of rows and columns of df1 and df2

print("df1", df1.count(), len(df1.columns))

print("df2", df2.count(), len(df2.columns))

df1 100 5
df2 100 4


In [7]:
df1.show(5)

+-----------+-----------+------+-----------+--------+
|customer_id|date_column|amount|description|location|
+-----------+-----------+------+-----------+--------+
|          1|   1/1/2022|  5000| Purchase A| Store A|
|          2|  15/2/2022|  1200| Purchase B| Store B|
|          3|  20/3/2022|   800| Purchase C| Store C|
|          4|  10/4/2022|  3000| Purchase D| Store D|
|          5|   5/5/2022|  6000| Purchase E| Store E|
+-----------+-----------+------+-----------+--------+
only showing top 5 rows



In [8]:
df2.show(5)

+-----------+----------------+-----+------+
|customer_id|transaction_date|value| notes|
+-----------+----------------+-----+------+
|          1|        1/1/2022| 1500|Note 1|
|          2|       15/2/2022| 2000|Note 2|
|          3|       20/3/2022| 1000|Note 3|
|          4|       10/4/2022| 2500|Note 4|
|          5|        5/5/2022| 1800|Note 5|
+-----------+----------------+-----+------+
only showing top 5 rows



#### Task 3: Add a new column to each dataframe

Add a new column named **year** to `df1` and **quarter** to `df2` representing the year and quarter of the data.

*Hint: use withColumn. Convert the date columns which are present as string to date before extracting the year and quarter information*




In [9]:
#from pyspark.sql.functions import year, quarter, to_date

#Add new column year to df1
df1 = df1.withColumn("year", year(to_date(df1["date_column"], 'd/M/yyyy')))

#Add new column quarter to df2
df2 = df2.withColumn("quarter", quarter(to_date(df2["transaction_date"], 'd/M/yyyy')))

# If I wanted instead to convert date given in format d/M/yyyy to date_format dd-MM-yyyy, it would be:
df2_2 = df2.withColumn("date", date_format(to_date(df2["transaction_date"], 'd/M/yyyy'), 'dd-MM-yyyy'))
df2_2.show(5)

+-----------+----------------+-----+------+-------+----------+
|customer_id|transaction_date|value| notes|quarter|      date|
+-----------+----------------+-----+------+-------+----------+
|          1|        1/1/2022| 1500|Note 1|      1|01-01-2022|
|          2|       15/2/2022| 2000|Note 2|      1|15-02-2022|
|          3|       20/3/2022| 1000|Note 3|      1|20-03-2022|
|          4|       10/4/2022| 2500|Note 4|      2|10-04-2022|
|          5|        5/5/2022| 1800|Note 5|      2|05-05-2022|
+-----------+----------------+-----+------+-------+----------+
only showing top 5 rows



#### Task 4: Rename columns in both dataframes

Rename the column **amount** to **transaction_amount** in `df1` and **value** to **transaction_value** in `df2`.

*Hint: Use withColumnRenamed*


In [10]:
#Rename df1 column amount to transaction_amount
df1 = df1.withColumnRenamed("amount", "transaction_amount")

#Rename df2 column value to transaction_value
df2 = df2.withColumnRenamed("value", "transaction_value")


#### Task 5: Drop unnecessary columns

Drop the columns **description** and **location** from `df1` and **notes** from `df2`.




In [14]:
#Drop columns description and location from df1
df1 = df1.drop("description","location")

#Drop column notes from df2
df2 = df2.drop("notes")


#### Task 6: Join dataframes based on a common column

Join `df1` and `df2` based on the common column **customer_id** and create a new dataframe named `joined_df`.




In [15]:
#join df1 and df2 based on common column customer_id

joined_df = df1.join(df2, on='customer_id', how='inner')

joined_df.show(5)

+-----------+-----------+------------------+----+----------------+-----------------+-------+
|customer_id|date_column|transaction_amount|year|transaction_date|transaction_value|quarter|
+-----------+-----------+------------------+----+----------------+-----------------+-------+
|          1|   1/1/2022|              5000|2022|        1/1/2022|             1500|      1|
|          2|  15/2/2022|              1200|2022|       15/2/2022|             2000|      1|
|          3|  20/3/2022|               800|2022|       20/3/2022|             1000|      1|
|          4|  10/4/2022|              3000|2022|       10/4/2022|             2500|      2|
|          5|   5/5/2022|              6000|2022|        5/5/2022|             1800|      2|
+-----------+-----------+------------------+----+----------------+-----------------+-------+
only showing top 5 rows



#### Task 7: Filter data based on a condition

Filter `joined_df` to include only transactions where "transaction_amount" is greater than 1000 and create a new dataframe named `filtered_df`.





In [16]:
# filter the dataframe for transaction amount > 1000

filtered_df = joined_df.filter(joined_df['transaction_amount']>1000)

filtered_df.show()

+-----------+-----------+------------------+----+----------------+-----------------+-------+
|customer_id|date_column|transaction_amount|year|transaction_date|transaction_value|quarter|
+-----------+-----------+------------------+----+----------------+-----------------+-------+
|          1|   1/1/2022|              5000|2022|        1/1/2022|             1500|      1|
|          2|  15/2/2022|              1200|2022|       15/2/2022|             2000|      1|
|          4|  10/4/2022|              3000|2022|       10/4/2022|             2500|      2|
|          5|   5/5/2022|              6000|2022|        5/5/2022|             1800|      2|
|          6|  10/6/2022|              4500|2022|       10/6/2022|             1200|      2|
|          8|  20/8/2022|              3500|2022|       20/8/2022|             3000|      3|
|         10| 30/10/2022|              1800|2022|      30/10/2022|             1200|      4|
|         11|  5/11/2022|              2200|2022|       5/11/2022|    

#### Task 8: Aggregate data by customer

Calculate the total transaction amount for each customer in `filtered_df` and display the result.

*Hint: Use sum from pyspark.sql.functions*


In [21]:

# group by customer_id and aggregate the sum of transaction amount

total_amount_per_customer = filtered_df.groupBy("customer_id").agg({"transaction_amount":"sum"})

#display the result

total_amount_per_customer.show()

+-----------+-----------------------+
|customer_id|sum(transaction_amount)|
+-----------+-----------------------+
|         31|                   3200|
|         85|                   1800|
|         78|                   1500|
|         34|                   1200|
|         81|                   5500|
|         28|                   2600|
|         76|                   2600|
|         27|                   4200|
|         91|                   3200|
|         22|                   1200|
|         93|                   5500|
|          1|                   5000|
|         52|                   2600|
|         13|                   4800|
|          6|                   4500|
|         16|                   2600|
|         40|                   2600|
|         94|                   1200|
|         57|                   5500|
|         54|                   1500|
+-----------+-----------------------+
only showing top 20 rows



#### Task 9: Write the result to a Hive table

Write `total_amount_per_customer` to a Hive table named **customer_totals**.


In [22]:
# Write total_amount_per_customer to a Hive table named customer_totals

total_amount_per_customer.write.mode("overwrite").saveAsTable("customer_totals")


#### Task 10: Write the filtered data to HDFS

Write `filtered_df` to HDFS in parquet format to a file named **filtered_data**.


In [23]:
#Write filtered_df to HDFS in parquet format file filtered_data.parquet

filtered_df.write.mode("overwrite").parquet("filtered_data.parquet")


#### Task 11: Add a new column based on a condition

Add a new column named **high_value** to `df1` indicating whether the transaction_amount is greater than 5000. When the value is greater than 5000, the value of the column should be **Yes**. When the value is less than or equal to 5000, the value of the column should be **No**.

*Hint: Use when and lit from pyspark.sql.functions


In [27]:
# Add new column with value indicating whether transaction amount is > 5000 or not

df1 = df1.withColumn('high_value', when(col("transaction_amount")>5000, "Yes").otherwise("No"))

df1.show()

+-----------+-----------+------------------+----+----------+
|customer_id|date_column|transaction_amount|year|high_value|
+-----------+-----------+------------------+----+----------+
|          1|   1/1/2022|              5000|2022|        No|
|          2|  15/2/2022|              1200|2022|        No|
|          3|  20/3/2022|               800|2022|        No|
|          4|  10/4/2022|              3000|2022|        No|
|          5|   5/5/2022|              6000|2022|       Yes|
|          6|  10/6/2022|              4500|2022|        No|
|          7|  15/7/2022|               200|2022|        No|
|          8|  20/8/2022|              3500|2022|        No|
|          9|  25/9/2022|               700|2022|        No|
|         10| 30/10/2022|              1800|2022|        No|
|         11|  5/11/2022|              2200|2022|        No|
|         12| 10/12/2022|               900|2022|        No|
|         13|  15/1/2023|              4800|2023|        No|
|         14|  20/2/2023

#### Task 12: Calculate the average transaction value per quarter

Calculate and display the average transaction value for each quarter in `df2` and create a new dataframe named `average_value_per_quarter` with column `avg_trans_val`.

*Hint: Use avg from pyspark.sql.functions*


In [28]:
#calculate the average transaction value for each quarter in df2

average_value_per_quarter = df2.groupBy("quarter").agg({"transaction_value":"average"})

#show the average transaction value for each quarter in df2

average_value_per_quarter.show()

+-------+----------------------+
|quarter|avg(transaction_value)|
+-------+----------------------+
|      1|     1111.111111111111|
|      3|    1958.3333333333333|
|      4|     816.6666666666666|
|      2|                1072.0|
+-------+----------------------+



#### Task 13: Write the result to a Hive table

Write `average_value_per_quarter` to a Hive table named **quarterly_averages**.


In [29]:
#Write average_value_per_quarter to a Hive table named quarterly_averages

average_value_per_quarter.write.mode("overwrite").saveAsTable("quarterly_averages")



<details>
    <summary>Click here for Solution</summary>

```python

#Write average_value_per_quarter to a Hive table named quarterly_averages

average_value_per_quarter.write.mode("overwrite").saveAsTable("quarterly_averages")

```

</details>


#### Task 14: Calculate the total transaction value per year

Calculate and display the total transaction value for each year in `df1` and create a new dataframe named `total_value_per_year` with column `total_transaction_val`.


In [31]:
# calculate the total transaction value for each year in df1.

total_value_per_year = df1.groupBy("year").agg({"transaction_amount":"sum"})

# show the total transaction value for each year in df1.

total_value_per_year.show()

+----+-----------------------+
|year|sum(transaction_amount)|
+----+-----------------------+
|2025|                  25700|
|2027|                  25700|
|2023|                  28100|
|2022|                  29800|
|2026|                  25700|
|2029|                  25700|
|2030|                   9500|
|2028|                  25700|
|2024|                  25700|
+----+-----------------------+



#### Task 15: Write the result to HDFS

Write `total_value_per_year` to HDFS in the CSV format to file named **total_value_per_year**.



In [ ]:
#Write total_value_per_year to HDFS in the CSV format

total_value_per_year.write.mode("overwrite").csv("total_value_per_year.csv")
